# This notebook displays how to create and query correlator data

## Import

In [ ]:
from typing import Union

import datetime
import pytz

from django.db import IntegrityError

from lattedb.project.formfac.models.data.correlator import (
    CorrelatorMeta,
    DiskCorrelatorH5Dset,
    TapeCorrelatorH5Dset,
)

Set a timezone for creating timezone aware objects

## Create a time for the given timezone

In [ ]:
us_timezones = sorted([timezone for timezone in pytz.all_timezones if "US" in timezone])
print([timezone.split("/")[-1] for timezone in us_timezones])

timezone = pytz.timezone("US/Michigan")

time = datetime.datetime(2020, 3, 11, 23, 59, 59, 1234, timezone)
time

## How to create a Disk or Tape entry for correlators

First create a meta object (this is the object which will eventually replaced by the actual meta tables)

In [ ]:
meta_infos = {"corr": "phi_qq", "configuration": 200, "source": "x22y21z20t19"}
corr_meta, created = CorrelatorMeta.objects.get_or_create(**meta_infos)
corr_meta

Next create a disk or Disk entry

In [ ]:
disk_infos = {
    "name": "corr.h5",
    "path": "/summit/path/to/file/",
    "dset": "phi_qq/cfg_199/src_x22y21z20t19/array",
    "exists": True,
    "machine": "Summit",
    "date_modified": time,
    "meta": corr_meta,
}
disk_meta, created = DiskCorrelatorH5Dset.objects.get_or_create(**disk_infos)
disk_meta

Tape creation works the same way with the difference that you should use `TapeCorrelatorH5Dset` instead of `DiskCorrelatorH5Dset`

## Bulk push disk

In [ ]:
meta_list = []
meta_tmp = meta_infos.copy()

for cfg in range(21, 30):
    meta_tmp["configuration"] = cfg
    # Create python object but do not push to db
    meta_list.append(CorrelatorMeta(**meta_tmp))

# Push to db
## Note: This only works if objects do not exist
try:
    meta_objs = CorrelatorMeta.objects.bulk_create(meta_list)
    print(meta_objs)
except IntegrityError:
    meta_objs = []
    print("At least one object already exists in db")

In [ ]:
disk_list = []
disk_tmp = disk_infos.copy()
    
for meta in meta_objs:
    disk_tmp["dset"] = f"phi_qq/cfg_{meta.configuration}/src_x22y21z20t19/array"
    # Pass *existing* python meta object to file info
    disk_tmp["meta"] = meta
    # Create python file object but do not push to db
    disk_list.append(DiskCorrelatorH5Dset(**disk_tmp))
    

# Push to db
## Note: This only works if objects do not exist
try:
    disk_objs = DiskCorrelatorH5Dset.objects.bulk_create(disk_list)
    print(disk_objs)
except IntegrityError:
    disk_objs = []
    print("At least one object already exists in db")

## Queries

### Find out if file exists somewhere for given meta

In [ ]:
def get_file_location(
    corr: str, configuration: int, source: str
) -> Union[DiskCorrelatorH5Dset, TapeCorrelatorH5Dset, None]:
    """Looks up if a given correlator can be found on disk or tape.
    
    Returns the corresponding object if found, else None.
    If both disk and tape object exists, return Disk object first.
    """
    obj = None
    meta = CorrelatorMeta.objects.filter(
        corr=corr, configuration=configuration, source=source
    ).first()

    if meta is not None:
        if hasattr(corr_meta, "disk") and corr_meta.disk.exists:
            obj = meta.disk
        elif hasattr(corr_meta, "tape") and corr_meta.tape.exists:
            obj = meta.tape

    return obj

In [ ]:
file_obj = get_file_location(**meta_infos)
print(file_obj.type)
print(file_obj.machine)
print(file_obj.file_address)
print(file_obj.dset)
print()

file_obj = get_file_location("mres", 200, "bla")
print(dset)